* [timestamp](#Timestamp)

In [1]:
import pandas as pd
import os
import time

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import decomposition
%matplotlib inline

In [28]:
data_folder = "held_in_csv/"

In [29]:
labels_df = pd.read_csv(data_folder + "Y_held_in.csv")
labels_df.head()

,SMID,Diagnosis2,Score
0,101378,Negative,0
1,103694,Positive,2
2,106002,Negative,0
3,107667,Negative,0
4,107776,Negative,0


# Timestamp

In [30]:
import datetime

Quick note: dayofweek 0=Monday, ..., Sunday=6

In [5]:
daytime = []
for day in ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]:
    daytime.append(day+" Day")
    daytime.append(day+" Night")
all_counts = pd.DataFrame(index=labels_df.SMID,columns=daytime+["sum","label","score"])

In [211]:
%%time

for k in range(len(labels_df)):

    SMID = labels_df.SMID[k]
    sentences_df = pd.read_csv(data_folder + str(SMID)+".csv")
    sentences_df.index = pd.DatetimeIndex(pd.to_datetime(sentences_df.TimeStamp, unit='ms')).tz_localize('UTC').tz_convert('US/Central')
    counts = sentences_df.groupby([lambda x: x.dayofweek, lambda x: x.hour]).count()

    df_counts = pd.DataFrame(0, index=daytime,columns=["count"])
    df_counts = df_counts.astype('int')

    for i in range(len(counts)):
        name = counts.iloc[i,:].name
        count = counts.iloc[i,:].Message
        week = name[0]*2
        if 8<=name[1]<=18:
            df_counts.iloc[week,0] = df_counts.iloc[week,0] + count
        else:
            df_counts.iloc[week+1,0] = df_counts.iloc[week+1,0] + count
    count_sum = df_counts.sum(axis=0)
    df_counts = df_counts/count_sum
    df_counts['sum'] = count_sum

    for time in daytime:
        all_counts.at[SMID,time] = df_counts.at[time,'count']
    all_counts.at[SMID,"sum"] = count_sum
    all_counts.at[SMID,"label"] = labels_df['Diagnosis2'][k]
    all_counts.at[SMID,"score"] = labels_df['Score'][k]

CPU times: user 2min 47s, sys: 12.6 s, total: 3min
Wall time: 2min 58s


In [210]:
df_counts/df_counts.sum(axis=0)

,count
Mon Day,0.084211
Mon Night,0.115789
Tue Day,0.063158
Tue Night,0.126316
Wed Day,0.126316
Wed Night,0.126316
Thu Day,0.042105
Thu Night,0.031579
Fri Day,0.063158
Fri Night,0.063158


In [201]:
all_counts.to_csv("counts.csv")

# 7 days

In [31]:

daytime = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
all_counts = pd.DataFrame(index=labels_df.SMID,columns=daytime+["sum","label","score"])

In [32]:
%%time

for k in range(len(labels_df)):

    SMID = labels_df.SMID[k]
    sentences_df = pd.read_csv(data_folder + str(SMID)+".csv")
    sentences_df.index = pd.DatetimeIndex(pd.to_datetime(sentences_df.TimeStamp, unit='ms')).tz_localize('UTC').tz_convert('US/Central')
    counts = sentences_df.groupby([lambda x: x.dayofweek, lambda x: x.hour]).count()

    df_counts = pd.DataFrame(0, index=daytime,columns=["count"])
    df_counts = df_counts.astype('int')

    for i in range(len(counts)):
        name = counts.iloc[i,:].name
        count = counts.iloc[i,:].Message
        week = name[0]
        df_counts.iloc[week,0] = df_counts.iloc[week,0] + count
        
    count_sum = df_counts.sum(axis=0)
    df_counts = df_counts/count_sum
    df_counts['sum'] = count_sum

    for time in daytime:
        all_counts.at[SMID,time] = df_counts.at[time,'count']
    all_counts.at[SMID,"sum"] = count_sum
    all_counts.at[SMID,"label"] = labels_df['Diagnosis2'][k]
    all_counts.at[SMID,"score"] = labels_df['Score'][k]

<decorator-gen-61>:2: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
<decorator-gen-61>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 26s, sys: 2.63 s, total: 1min 28s
Wall time: 1min 30s


In [33]:
all_counts.to_csv("counts_7.csv")

In [35]:
all_counts.head()

,Mon,Tue,Wed,Thu,Fri,Sat,Sun,sum,label,score
SMID,,,,,,,,,,
101378,0.057514,0.068646,0.291280,0.155844,0.168831,0.066790,0.191095,count 539\ndtype: int64,Negative,0
103694,0.166939,0.133209,0.167329,0.100647,0.136948,0.111864,0.183065,count 12837\ndtype: int64,Positive,2
106002,0.153758,0.150040,0.142693,0.151589,0.155085,0.097194,0.149641,count 22594\ndtype: int64,Negative,0
107667,0.135276,0.142040,0.158689,0.248179,0.080125,0.104058,0.131634,count 1922\ndtype: int64,Negative,0
107776,0.121941,0.181134,0.148504,0.131981,0.095378,0.139092,0.181970,count 4781\ndtype: int64,Negative,0


# weekdays

In [45]:
daytime = ["weekdays","weekends"]
all_counts = pd.DataFrame(index=labels_df.SMID,columns=daytime+["sum","label","score"])

In [46]:
%%time

for k in range(len(labels_df)):

    SMID = labels_df.SMID[k]
    sentences_df = pd.read_csv(data_folder + str(SMID)+".csv")
    sentences_df.index = pd.DatetimeIndex(pd.to_datetime(sentences_df.TimeStamp, unit='ms')).tz_localize('UTC').tz_convert('US/Central')
    counts = sentences_df.groupby([lambda x: x.dayofweek, lambda x: x.hour]).count()

    df_counts = pd.DataFrame(0, index=daytime,columns=["count"])
    df_counts = df_counts.astype('int')

    for i in range(len(counts)):
        name = counts.iloc[i,:].name
        count = counts.iloc[i,:].Message
        week = name[0]
        if week <4:
            df_counts.iloc[0,0] = df_counts.iloc[0,0] + count
        else:
            df_counts.iloc[1,0] = df_counts.iloc[1,0] + count
        
    count_sum = df_counts.sum(axis=0)
    df_counts = df_counts/count_sum
    df_counts['sum'] = count_sum

    for time in daytime:
        all_counts.at[SMID,time] = df_counts.at[time,'count']
    all_counts.at[SMID,"sum"] = count_sum
    all_counts.at[SMID,"label"] = labels_df['Diagnosis2'][k]
    all_counts.at[SMID,"score"] = labels_df['Score'][k]

<decorator-gen-61>:2: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
<decorator-gen-61>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 26s, sys: 1.23 s, total: 1min 27s
Wall time: 1min 27s


In [47]:
all_counts.to_csv("counts_2.csv")

# Linear regression

In [48]:
all_counts = pd.read_csv("counts_2.csv",index_col="SMID")
all_counts = all_counts.fillna(0)

In [37]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

In [9]:
X_train, X_test, y_train, y_test = train_test_split(all_counts.iloc[:,0:14].values, 
                                                    all_counts['label'].values, test_size=0.2, stratify=all_counts['label'].values)

n_samples / (n_classes * np.bincount(y))

In [50]:
X = all_counts.iloc[:,0:2].values
y = all_counts['label'].values

In [273]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

lm = linear_model.LogisticRegression(class_weight='balanced')
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

print(classification_report(y_test, predictions))
print(classification_report(y_train, lm.predict(X_train)))

              precision    recall  f1-score   support

    Negative       0.78      0.64      0.70       103
    Positive       0.24      0.39      0.30        31

    accuracy                           0.58       134
   macro avg       0.51      0.51      0.50       134
weighted avg       0.65      0.58      0.61       134

              precision    recall  f1-score   support

    Negative       0.78      0.66      0.71       411
    Positive       0.26      0.39      0.31       125

    accuracy                           0.60       536
   macro avg       0.52      0.52      0.51       536
weighted avg       0.66      0.60      0.62       536



/Users/elisa/miniconda3/envs/lucky/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [39]:
from sklearn.model_selection import cross_val_predict,cross_val_score

In [52]:
lm = linear_model.LogisticRegression(class_weight='balanced')
predictions = cross_val_predict(lm, X, y, cv=5)
print(classification_report(y, predictions))

lm = linear_model.LogisticRegression()
predictions = cross_val_predict(lm, X, y, cv=5)
print(classification_report(y, predictions))


              precision    recall  f1-score   support

    Negative       0.77      0.80      0.79       514
    Positive       0.25      0.22      0.24       156

    accuracy                           0.67       670
   macro avg       0.51      0.51      0.51       670
weighted avg       0.65      0.67      0.66       670

              precision    recall  f1-score   support

    Negative       0.77      1.00      0.87       514
    Positive       0.00      0.00      0.00       156

    accuracy                           0.77       670
   macro avg       0.38      0.50      0.43       670
weighted avg       0.59      0.77      0.67       670



/home/i/miniconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
